In [1]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [2]:
trdata = ImageDataGenerator()
tsdata = ImageDataGenerator()


In [5]:
traindata = trdata.flow_from_directory(directory="./datasets/flowers",target_size=(224,224), class_mode="sparse")
testdata = tsdata.flow_from_directory(directory="./datasets/flowers_small", target_size=(224,224), class_mode="sparse")

Found 3817 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [17]:
import tensorflow as tf
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

102967424/102967424 [==============================] - 3s 0us/step


In [18]:
len(model.layers)

177

In [19]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [20]:
for layers in (model.layers)[:143]:
    layers.trainable = False

In [21]:
X= model.layers[-2].output
predictions = Dense(5, activation="softmax")(X)
model_final = Model(model.input, predictions)


In [22]:
model_final.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizers.SGD(learning_rate=0.001, momentum=0.9), metrics=["accuracy"])

In [23]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
metric = 'val_accuracy'
checkpoint = ModelCheckpoint("resnet50_1.h5", monitor=metric, verbose=2, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)
early = EarlyStopping(monitor='metric', min_delta=0.02, patience=5, verbose=1, mode='auto')


In [24]:
model_final.fit_generator(generator= traindata, steps_per_epoch= 1, epochs= 10, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])
model_final.save_weights("resnet50_1.h5")

Epoch 1/10


C:\Users\dell\AppData\Local\Temp\ipykernel_1076\2801998838.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_final.fit_generator(generator= traindata, steps_per_epoch= 1, epochs= 10, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])


1/1 [==============================] - 9s 9s/step - loss: 1.7599 - accuracy: 0.2812 - val_loss: 2.0084 - val_accuracy: 0.2188
Epoch 2/10
1/1 [==============================] - 5s 5s/step - loss: 1.8034 - accuracy: 0.3125 - val_loss: 1.7045 - val_accuracy: 0.3125
Epoch 3/10
1/1 [==============================] - 5s 5s/step - loss: 1.7108 - accuracy: 0.1250 - val_loss: 1.6842 - val_accuracy: 0.2812
Epoch 4/10
1/1 [==============================] - 4s 4s/step - loss: 1.5636 - accuracy: 0.3125 - val_loss: 1.6453 - val_accuracy: 0.3750
Epoch 5/10
1/1 [==============================] - 5s 5s/step - loss: 1.7197 - accuracy: 0.2500 - val_loss: 1.9866 - val_accuracy: 0.2500
Epoch 6/10
1/1 [==============================] - 4s 4s/step - loss: 1.6360 - accuracy: 0.2812 - val_loss: 1.8636 - val_accuracy: 0.3125
Epoch 7/10
1/1 [==============================] - 5s 5s/step - loss: 1.4418 - accuracy: 0.4688 - val_loss: 1.4602 - val_accuracy: 0.5312
Epoch 8/10
1/1 [==============================] - 5s